# Memcache for large files

Design a cache for large objects (> 1MB) on top of memcache.

You have access to the following memcache methods:
    
    memcache.get(key)
    memcache.set(key, value)
    

The interface for the BigCache should be similarly simple from the users perspective.  We should have the basic get and get and set operations, and additionally a delete operation:

    BigCache.get(key)
    BigCache.set(key, value)
    BigCache.delete(key)

Underneath, we should partition our memcache into blocks of the maximum allowed size, 1MB, emumerate them, then regulate access with a set.  So, if we have a memcache of 1GB, we have 1000 available blocks, from 0 to 999.  

We'll need to mainain an arrays of available blocks, and a map from key to used blocks associated with that key.

So, as a class it might look like this:

In [ ]:
class obj:
    
    def __init__(self, value, size):
        self.value = value
        self.size = size # in MB



class memcache:
    
    def __init__(self, size):
        self.size = size # in MB
        self.cache = {}
    
    def get(self, key):
        return self.cache[key]
    
    def set(self, key, value):
        self.cache[key] = value


class BigCache:
    
    def __init__(self, mem):
        blocks = mem.size
        self.avaiable_blocks = list(range(0, blocks))
        self.mem = mem
        self.cache = {} # map from the object key, to the block inices
    
    def get(self, key):
        if key in self.cache:
            partitions = [self.mem.get(idx) for idx in self.cache[key]]
            return _reconstruct_object(partitions)
        else:
            raise Exception('Key not found')
        
        
    
    def set(self, key, value):
        if len(self.avaiable_blocks) >= _get_necessary_blocks(value):
            self.cache[key] = []
            partitioned = _partition(value)
            for i in partitioned:
                idx = self.avaiable_blocks.pop()
                self.cache[key].append(idx)
                self.mem[idx] = i
        else:
            raise Exception('cache is full')
        
    
    def delete(self, key):
        for idx in self.cache[key]:
            # set blocks associated with this key to available
            # we might want to explicitly remove them, but our memcache api doesn't allow this
            self.avaiable_blocks.append(idx)
    
    def _get_necessary_blocks(self,value):
        blocks = value.size//1
        if (value.size % 1) > 0:
            blocks += 1
        return blocks
    
    def _partition(self, value):
        remainder = value.size % 1
        return [obj(value.value,1) for i in range(value.size//1)] + [(obj(v,remainder))]
    
    def _reconstruct_object(self, partitions):
        size = len(partitions) + partitions[-1].size
        return obj(partitions[0].value, size)
